## 1. Import Modules

In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from config.secret_keys import OPENAI_API_KEY
import pandas as pd
from tqdm import tqdm

## 2. Load Raw Data

In [4]:
texts = pd.read_csv("./data/texts.csv")

texts = list(texts["text"])

len(texts)

224

## 3. Generate Summarization/Expansion Dataset

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0
)

def summarize_text(llm, text):
    messages = [
        SystemMessage(content="너는 전문적인 요약가야. 주어진 텍스트를 간결하게 요약해 줘."),
        HumanMessage(content=text)
    ]
    
    response = llm(messages)

    return response.content

def expansion_text(llm, text):
    messages = [
        SystemMessage(content="너는 전문적인 문장 분량 확장자야. 주어진 텍스트의 분량을 사실적으로 늘려줘."),
        HumanMessage(content=text)
    ]
    
    response = llm(messages)

    return response.content

In [ ]:
summary_dataset = []

for text in tqdm(texts):
    summary_dataset.append([text, summarize_text(llm, text)])

summary_dataset = pd.DataFrame(summary_dataset, columns=["text", "target"])

for idx, text in enumerate(summary_dataset["text"]):
    summary_dataset.loc[idx, "text"] = text + " 이 텍스트를 요약해줘."

100%|██████████| 224/224 [09:46<00:00,  2.62s/it]


In [ ]:
expansion_dataset = []

for text in tqdm(texts):
    expansion_dataset.append([text, expansion_text(llm, text)])

expansion_dataset = pd.DataFrame(expansion_dataset, columns=["text", "target"])

for idx, text in enumerate(expansion_dataset["text"]):
    expansion_dataset.loc[idx, "text"] = text + " 이 텍스트를 확장해줘."

100%|██████████| 224/224 [26:47<00:00,  7.17s/it]


In [84]:
summary_dataset.to_csv("./data/summary_dataset.csv", index=False, encoding="utf-8")
expansion_dataset.to_csv("./data/expansion_dataset.csv", index=False, encoding="utf-8")

## 4. Split Train/Test Dataset

In [6]:
summary_dataset = pd.read_csv("./data/summary_dataset.csv")
expansion_dataset = pd.read_csv("./data/expansion_dataset.csv")

### 4-1. Suumarization Data EDA

In [13]:
summary_dataset["text"].describe()

count                                                   224
unique                                                  223
top       현대자동차그룹의 계열사로, 그룹 내 종합물류 업무를 전담한다. 2001년 한국로지텍...
freq                                                      2
Name: text, dtype: object

### 4-2. Split Data

In [7]:
from sklearn.model_selection import train_test_split

x_train_summary, x_test_summary, y_train_summary, y_test_summary = train_test_split(
    summary_dataset["text"], summary_dataset["target"], test_size=0.09, random_state=42
)

x_train_expansion, x_test_expansion, y_train_expansion, y_test_expansion = train_test_split(
    expansion_dataset["text"], expansion_dataset["target"], test_size=0.09, random_state=42
)

In [8]:
x_train_summary = pd.DataFrame(x_train_summary)
y_train_summary = pd.DataFrame(y_train_summary)
x_test_summary = pd.DataFrame(x_test_summary)
y_test_summary = pd.DataFrame(y_test_summary)

x_train_expansion = pd.DataFrame(x_train_expansion)
y_train_expansion = pd.DataFrame(y_train_expansion)
x_test_expansion = pd.DataFrame(x_test_expansion)
y_test_expansion = pd.DataFrame(y_test_expansion)

### 4-3. Train Dataset EDA

In [9]:
x_train_summary

,text
18,삼성바이오에피스는 2019년 흑자 전환에 성공한 후 매년 두 자릿수의 영업이익률을 ...
79,한국보건산업진흥원이 발간한 ‘글로벌 제약바이오 산업동향’ 보고서에 따르면 전 세계 ...
141,시장이 큰 만큼 스텔라라 바이오시밀러 시장에 뛰어든 기업 또한 국내외로 많다. 국내...
147,동아에스티의 해외 매출 중 가장 큰 비중을 차지하는 것은 ‘박카스’다. 2023년 ...
143,동아에스티가 2013년부터 메이지세이카파마와 공동 개발해 온 스텔라라 바이오시밀러 ...
...,...
106,컬러강판 시장의 경쟁이 격화되고 있다. 철강에 디자인을 적용해 다양한 소재의 무늬와...
14,그동안 축적된 기술력을 토대로 최근 급격하게 성장하고 있는 OLED 디스플레이에 적...
92,"전자 제품의 판매는 사회문화적 변화를 비롯해 미세먼지, 황사 등 외부 환경 변화에 ..."
179,우아한형제들은 2010년 6월 배달 앱 ‘배달의민족’을 론칭했다. 현재 국내 1등 ...


## 5. Transform Dataset into Fine-Tuning Prompt

In [87]:
def generate_prompt(texts, targets):
    output_texts = []
    for text, target in zip(texts["text"], targets["target"]):
        prompt = f"### Instruction: {text}\n\n### Response: {target}<eos>"
        output_texts.append(prompt)
    return output_texts

In [88]:
train_summary = generate_prompt(x_train_summary, y_train_summary)
train_expansion = generate_prompt(x_train_expansion, y_train_expansion)
test_summary = generate_prompt(x_test_summary, y_test_summary)
test_expansion = generate_prompt(x_test_expansion, y_test_expansion)

train_summary = pd.DataFrame(train_summary, columns=["text"])
train_expansion = pd.DataFrame(train_expansion, columns=["text"])
test_summary = pd.DataFrame(test_summary, columns=["text"])
test_expansion = pd.DataFrame(test_expansion, columns=["text"])

train_dataset = pd.concat([train_summary, train_expansion], axis=0)
test_dataset = pd.concat([test_summary, test_expansion], axis=0)

In [89]:
train_dataset

,text
0,### Instruction: 삼성바이오에피스는 2019년 흑자 전환에 성공한 후 ...
1,### Instruction: 한국보건산업진흥원이 발간한 ‘글로벌 제약바이오 산업동...
2,### Instruction: 시장이 큰 만큼 스텔라라 바이오시밀러 시장에 뛰어든 ...
3,### Instruction: 동아에스티의 해외 매출 중 가장 큰 비중을 차지하는 ...
4,### Instruction: 동아에스티가 2013년부터 메이지세이카파마와 공동 개...
...,...
198,### Instruction: 컬러강판 시장의 경쟁이 격화되고 있다. 철강에 디자인...
199,### Instruction: 그동안 축적된 기술력을 토대로 최근 급격하게 성장하고...
200,### Instruction: 전자 제품의 판매는 사회문화적 변화를 비롯해 미세먼지...
201,### Instruction: 우아한형제들은 2010년 6월 배달 앱 ‘배달의민족’...


In [90]:
train_dataset.to_csv("./data/train_dataset.csv", index=False, encoding='utf-8')
test_dataset.to_csv("./data/test_dataset.csv", index=False, encoding='utf-8')